In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dl-kaggle-dataset/cleaned_train_x.csv
/kaggle/input/dl-kaggle-dataset/cleaned_val_x.csv
/kaggle/input/dl-kaggle-dataset/Train.py
/kaggle/input/dl-kaggle-dataset/cleaned_test_x.csv
/kaggle/input/dl-kaggle-dataset/train_y.csv
/kaggle/input/dl-kaggle-dataset/train_x.csv
/kaggle/input/dl-kaggle-dataset/test_x.csv
/kaggle/input/dl-kaggle-dataset/glove.840B.300d.txt
/kaggle/input/dl-kaggle-dataset/DataPreprocessing.py
/kaggle/input/dl-kaggle-dataset/val_x.csv
/kaggle/input/dl-kaggle-dataset/val_y.csv
/kaggle/input/dl-kaggle-dataset/rnn_baseline.py
/kaggle/input/dl-kaggle-dataset/cleanwords.txt
/kaggle/input/dl-kaggle-dataset/DataLoader.py


In [2]:
from joblib import Parallel, delayed
from textblob import TextBlob
from textblob.translate import NotTranslated

import argparse
import os
import pandas as pd

NAN_WORD = "_NAN_"

def translate(comment, language):
    if hasattr(comment, "decode"):
        comment = comment.decode("utf-8")

    text = TextBlob(comment)
    
    try:
        text = text.translate(from_lang='en', to=language)
        text = text.translate(from_lang=language, to='en')
    
    except NotTranslated:
        pass
    
    return str(text)

In [3]:
from pygtrans import Translate

In [5]:
original = "Wikipedia is full of fools. Who takes money and makes people work for free? Wikipedia!!! You might as well ban me, you fool. What's taking so long? Wiki is a stupid place, it's Jimbo's Cult."
# fr_trans = translate(original, 'fr')

In [9]:
client = Translate()

text = client.translate(original, target='fr')
text = client.translate(text.translatedText, target='en')

In [10]:
print(original)
print(text.translatedText)

Wikipedia is full of fools. Who takes money and makes people work for free? Wikipedia!!! You might as well ban me, you fool. What's taking so long? Wiki is a stupid place, it's Jimbo's Cult.
Wikipedia is full of idiots. Who takes money and makes people work for free? Wikipedia!!! You might as well ban me, you idiot. What&#39;s taking so long? Wiki is a stupid place, it&#39;s Jimbo&#39;s Cult.


In [4]:
print(original)
print(fr_trans)

Wikipedia is full of fools. Who takes money and makes people work for free? Wikipedia!!! You might as well ban me, you fool. What's taking so long? Wiki is a stupid place, it's Jimbo's Cult.
Wikipedia is full of fools. Who takes money and makes people work for free? Wikipedia !!! You could so well forbid me, you are silly. What takes so long? Wiki is a stupid place, it is the cult of Jimbo.


In [ ]:
from tqdm import tqdm
import time

def data_augmentation(train_data, languages, batch_size=100, pause_duration=100):
    extended_data = []
    sentences = train_data["string"].fillna(NAN_WORD).values
    
    for language in languages:
        print(f'Translate comments using {language}')
        
        translated_data = []
        total_batches = len(sentences) // batch_size

        for batch_num in range(total_batches):
            start_idx = batch_num * batch_size
            end_idx = (batch_num + 1) * batch_size

            batch_sentences = sentences[start_idx:end_idx]
            batch_translated_comments = []

            print(f'Translating using {language} - Batch {batch_num + 1}/{total_batches}')
            
            for i, comment in enumerate(tqdm(batch_sentences, desc=f'Translating using {language} - Batch {batch_num + 1}/{total_batches}')):
                time.sleep(0.1)
                translated_comment = translate(comment, language)
                batch_translated_comments.append(translated_comment)
                
            translated_data.extend(batch_translated_comments)
            time.sleep(pause_duration)  # Pause between batches
        
        extended_data.append(translated_data)
        
    return extended_data

In [ ]:
### Import original data
train_x = pd.read_csv(os.path.join(dirname, 'train_x.csv'))
train_y = pd.read_csv(os.path.join(dirname, 'train_y.csv'))
val_x = pd.read_csv(os.path.join(dirname, 'val_x.csv'))
val_y = pd.read_csv(os.path.join(dirname, 'val_y.csv'))
train = pd.concat([train_x, train_y], axis = 1)
val = pd.concat([val_x, val_y], axis = 1)
train = train[train.black == 1]
val = val[val.black == 1]

In [ ]:
val

In [ ]:
# TTA
languages = ['fr']
# extended_train_x = data_augmentation(train, languages)
extended_val_x = data_augmentation(val, languages)

# train_sentences = extended_train_x[0]
val_sentences = extended_val_x[0]

# augmented_train_x = pd.DataFrame({"string": train_sentences})
augmented_val_x = pd.DataFrame({"string": val_sentences})

In [44]:
all_train_sentences.append(augmented_train_x)

In [45]:
all_train_sentences

[                                               string
 0                            Even here ....... black!
 1   "Come on blacks and whites in the throat of th...
 2   Thank you, Monica. Unfortunately, it is easier...
 3   (from Internets)\n\nWhite: "Blacks should prot...
 4   The first slave owner in America was black !!!...
 ..                                                ...
 95  Do you mean this dual laughable standard where...
 96  You confuse "a" problem with "the problem". Ce...
 97  Because fanaticism only comes on one side, the...
 98  His state of mind is everything. A good attty ...
 99  Statistics on black crime, indigenous graduati...
 
 [100 rows x 1 columns],
                                                string
 0   Why should they scare you? You are not black o...
 1   Boy, you know who spent a lot of his money thi...
 2   Yes, boss Coghill, we certainly do not want on...
 3   Another black racist raised on the history of ...
 4   Sorry guy, each big city has a st

In [15]:
copied_data_y = pd.concat([test_data_y.copy(), test_data_y.copy()], axis=0)
result_data = pd.concat([test_data_y, copied_data_y], axis=0)
result_data = result_data.reset_index(drop = True)

In [16]:
result_data

,male,female,LGBTQ,christian,muslim,other_religions,black,white,identity_any,severe_toxicity,obscene,threat,insult,identity_attack,sexual_explicit,y,from_source_domain
0,0,0,0,0,0,0,1,1,1,0,0,0,1,1,0,1,0
1,0,1,0,0,0,0,0,0,1,0,1,0,0,0,1,1,0
2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
5,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
9,0,0,0,0,1,0,0,0,1,0,0,0,1,1,0,1,0
